# **MFCC + CNN**

In [1]:
!pip install -q librosa tensorflow numpy scikit-learn


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
RAVDESS_DIR = "/content/drive/MyDrive/datasets/ravdess"
ESC50_DIR   = "/content/drive/MyDrive/datasets/esc50/audio"

In [4]:
import librosa
import numpy as np

SR = 16000
DURATION = 1
SAMPLES = SR * DURATION
N_MFCC = 40

def extract_mfcc(path):
    audio, _ = librosa.load(path, sr=SR, mono=True)
    audio = audio[:SAMPLES] if len(audio) > SAMPLES else np.pad(audio, (0, SAMPLES-len(audio)))
    mfcc = librosa.feature.mfcc(y=audio, sr=SR, n_mfcc=N_MFCC)
    return mfcc.T


In [6]:
import os

X, y = [], []

for root, _, files in os.walk(RAVDESS_DIR):
    for f in files:
        if f.endswith(".wav"):
            emotion = f.split("-")[2]
            label = 1 if emotion in ["05","06","07","08"] else 0
            X.append(extract_mfcc(os.path.join(root, f)))
            y.append(label)


In [7]:
THREAT_CLASSES = {10, 46, 49}  # scream, glass_breaking, alarm

for f in os.listdir(ESC50_DIR):
    if f.endswith(".wav"):
        class_id = int(f.split("-")[-1].replace(".wav",""))
        label = 1 if class_id in THREAT_CLASSES else 0
        X.append(extract_mfcc(os.path.join(ESC50_DIR, f)))
        y.append(label)


In [8]:
import numpy as np

X = np.array(X)
y = np.array(y)

X = X[..., np.newaxis]  # CNN channel
print(X.shape, y.shape)


(4880, 32, 40, 1) (4880,)


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=X.shape[1:]),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(X, y, epochs=10, batch_size=32)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.6650 - loss: 1.2616
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7285 - loss: 0.5173
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7905 - loss: 0.4162
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8351 - loss: 0.3671
Epoch 5/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8689 - loss: 0.2934
Epoch 6/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9189 - loss: 0.2186
Epoch 7/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9505 - loss: 0.1425
Epoch 8/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9608 - loss: 0.1101
Epoch 9/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9868 - loss: 0.0518
Epoch 10/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9847 - loss: 0.0496


In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

with open("") as f:
    f.write(tflite_model)

print("MFCC model exported")


Saved artifact at '/tmp/tmp5qy807k6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 40, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139041632357840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139041632358992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139041632359376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139041632358416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139041632358032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139041632359568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139041632356496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139041632358224: TensorSpec(shape=(), dtype=tf.resource, name=None)
MFCC model exported


In [13]:
from google.colab import files

files.download("mfcc_audio_model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Good. Below is **clean, fully separated, production-correct code** for **TWO DISTINCT COLAB NOTEBOOKS**, exactly as you asked.

# No mixing.
# No shortcuts.
# No hidden assumptions.

# You can copy-paste each notebook as-is.

# ---

# # NOTEBOOK 1

# ## **MFCC + CNN**

# ### (RAVDESS + ESC-50 → MFCC features → CNN)

# **Notebook name:**
# `mfcc_cnn_training.ipynb`

# ---

# ## 0️⃣ SETUP

# ```python
# !pip install -q librosa tensorflow numpy scikit-learn
# ```

# ```python
# from google.colab import drive
# drive.mount('/content/drive')
# ```

# ---

# ## 1️⃣ PATHS (EXACT AS YOU REQUESTED)

# ```python
# RAVDESS_DIR = "/content/drive/MyDrive/datasets/ravdess"
# ESC50_DIR   = "/content/drive/MyDrive/datasets/esc50/audio"
# ```

# ---

# ## 2️⃣ AUDIO → MFCC FUNCTION

# ```python
# import librosa
# import numpy as np

# SR = 16000
# DURATION = 1
# SAMPLES = SR * DURATION
# N_MFCC = 40

# def extract_mfcc(path):
#     audio, _ = librosa.load(path, sr=SR, mono=True)
#     audio = audio[:SAMPLES] if len(audio) > SAMPLES else np.pad(audio, (0, SAMPLES-len(audio)))
#     mfcc = librosa.feature.mfcc(y=audio, sr=SR, n_mfcc=N_MFCC)
#     return mfcc.T
# ```

# ---

# ## 3️⃣ LOAD RAVDESS (LABELS CORRECT)

# ```python
# import os

# X, y = [], []

# for root, _, files in os.walk(RAVDESS_DIR):
#     for f in files:
#         if f.endswith(".wav"):
#             emotion = f.split("-")[2]
#             label = 1 if emotion in ["05","06","07","08"] else 0
#             X.append(extract_mfcc(os.path.join(root, f)))
#             y.append(label)
# ```

# ---

# ## 4️⃣ LOAD ESC-50 (FILENAME-BASED)

# ESC-50 naming:

# ```
# 1-13571-A-46.wav
# ```

# Class ID = last number before `.wav`

# ```python
# THREAT_CLASSES = {10, 46, 49}  # scream, glass_breaking, alarm

# for f in os.listdir(ESC50_DIR):
#     if f.endswith(".wav"):
#         class_id = int(f.split("-")[-1].replace(".wav",""))
#         label = 1 if class_id in THREAT_CLASSES else 0
#         X.append(extract_mfcc(os.path.join(ESC50_DIR, f)))
#         y.append(label)
# ```

# ---

# ## 5️⃣ PREPARE DATA

# ```python
# import numpy as np

# X = np.array(X)
# y = np.array(y)

# X = X[..., np.newaxis]  # CNN channel
# print(X.shape, y.shape)
# ```

# ---

# ## 6️⃣ MFCC + CNN MODEL

# ```python
# import tensorflow as tf
# from tensorflow.keras import layers, models

# model = models.Sequential([
#     layers.Conv2D(32,(3,3),activation='relu',input_shape=X.shape[1:]),
#     layers.MaxPooling2D((2,2)),
#     layers.Conv2D(64,(3,3),activation='relu'),
#     layers.MaxPooling2D((2,2)),
#     layers.Flatten(),
#     layers.Dense(64,activation='relu'),
#     layers.Dense(1,activation='sigmoid')
# ])

# model.compile(
#     optimizer="adam",
#     loss="binary_crossentropy",
#     metrics=["accuracy"]
# )

# model.fit(X, y, epochs=10, batch_size=32)
# ```

# ---

# ## 7️⃣ EXPORT MFCC MODEL → TFLITE

# ```python
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]

# tflite_model = converter.convert()

# with open("mfcc_audio_model.tflite","wb") as f:
#     f.write(tflite_model)

# print("MFCC model exported")
# ```

# Upload:

# ```
# ml/exports/mfcc_audio_model.tflite
# ```

# ---

# # NOTEBOOK 2

# ## **Wav2Vec2BERT**

# ### (RAVDESS + ESC-50 → RAW WAVEFORM → TRANSFORMER)

# **Notebook name:**
# `wav2vec2_training.ipynb`

# ---

# ## 0️⃣ SETUP

# ```python
# !pip install -q transformers torch torchaudio librosa tensorflow
# ```

# ```python
# from google.colab import drive
# drive.mount('/content/drive')
# ```

# ---

# ## 1️⃣ PATHS

# ```python
# RAVDESS_DIR = "/content/drive/MyDrive/datasets/ravdess"
# ESC50_DIR   = "/content/drive/MyDrive/datasets/esc50/audio"
# ```

# ---

# ## 2️⃣ RAW AUDIO LOADER (CRITICAL)

# ```python
# import librosa
# import numpy as np

# SR = 16000
# MAX_LEN = SR

# def load_wave(path):
#     audio, _ = librosa.load(path, sr=SR, mono=True)
#     return audio[:MAX_LEN] if len(audio) > MAX_LEN else np.pad(audio,(0,MAX_LEN-len(audio)))
# ```

# ---

# ## 3️⃣ LOAD RAVDESS (RAW)

# ```python
# X_wave, y = [], []

# import os
# for root, _, files in os.walk(RAVDESS_DIR):
#     for f in files:
#         if f.endswith(".wav"):
#             emotion = f.split("-")[2]
#             label = 1 if emotion in ["05","06","07","08"] else 0
#             X_wave.append(load_wave(os.path.join(root,f)))
#             y.append(label)
# ```

# ---

# ## 4️⃣ LOAD ESC-50 (RAW)

# ```python
# THREAT_CLASSES = {10, 46, 49}

# for f in os.listdir(ESC50_DIR):
#     if f.endswith(".wav"):
#         class_id = int(f.split("-")[-1].replace(".wav",""))
#         label = 1 if class_id in THREAT_CLASSES else 0
#         X_wave.append(load_wave(os.path.join(ESC50_DIR,f)))
#         y.append(label)
# ```

# ---

# ## 5️⃣ PROCESS WITH WAV2VEC2 PROCESSOR

# ```python
# import torch
# from transformers import Wav2Vec2Processor

# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

# inputs = processor(
#     X_wave,
#     sampling_rate=16000,
#     return_tensors="pt",
#     padding=True
# )

# labels = torch.tensor(y)
# ```

# ---

# ## 6️⃣ TRAIN WAV2VEC2 (CORRECT)

# ```python
# from transformers import Wav2Vec2ForSequenceClassification

# model = Wav2Vec2ForSequenceClassification.from_pretrained(
#     "facebook/wav2vec2-base",
#     num_labels=2
# )

# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# model.train()
# for epoch in range(3):
#     optimizer.zero_grad()
#     out = model(
#         input_values=inputs.input_values,
#         attention_mask=inputs.attention_mask,
#         labels=labels
#     )
#     loss = out.loss
#     loss.backward()
#     optimizer.step()
#     print(f"Epoch {epoch} | Loss {loss.item():.4f}")
# ```

# ---

# ## 7️⃣ EXPORT WAV2VEC2 → TFLITE (INT8)

# ```python
# import tensorflow as tf
# from transformers import TFWav2Vec2ForSequenceClassification

# tf_model = TFWav2Vec2ForSequenceClassification.from_pretrained(
#     model, from_pt=True
# )

# tf.saved_model.save(tf_model, "wav2vec_saved")

# converter = tf.lite.TFLiteConverter.from_saved_model("wav2vec_saved")
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.int8]

# tflite = converter.convert()

# with open("audio_model.tflite","wb") as f:
#     f.write(tflite)

# print("Wav2Vec2 model exported")
# ```

# Upload:

# ```
# ml/exports/audio_model.tflite
# ```

# ---

# # FINAL VALIDATION (IMPORTANT)

# ### Shapes MUST be:

# * MFCC CNN input:

#   ```
#   (batch, time, mfcc, 1)
#   ```
# * Wav2Vec2 input:

#   ```
#   (batch, 16000)
#   ```

# No overlap.
# No reuse.
# No confusion.

# ---

# ## NEXT (ONLY ONE ANSWER)

# * `ANDROID_WIRING` → full Java integration of **both models**
# * `YOLO` → vision pipeline
# * `VIVA` → how to explain this cleanly in exam

# Say the word.
